# Débruitage par minimisation de la variation totale

## Chargement des modules et des fonctions de base

In [1]:
# import modules
import numpy as np
from math import *
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as pl
from IPython import display
import time

In [16]:
# 2D discrete gradient 
def grad(u):
    ny,nx = u.shape
    Gx = np.zeros(u.shape)
    Gy = np.zeros(u.shape)
    Gx[:,0:nx-1] = u[:,1:nx] - u[:,0:nx-1]
    Gy[0:ny-1,:] = u[1:ny,:] - u[0:ny-1,:]
    return Gx,Gy

In [18]:
# divergence of a 2D discrete field
def div(px,py):
    ny,nx = px.shape
    # process the first component (px) of the input field
    div_x = np.zeros(px.shape)
    div_x[:,1:nx-1] = px[:,1:nx-1] - px[:,0:nx-2]
    div_x[:,0] = px[:,0]
    div_x[:,nx-1] = -px[:,nx-2]
    # process the second component (py) of the input field
    div_y = np.zeros(px.shape)
    div_y[1:ny-1,:] = py[1:ny-1,:] - py[0:ny-2,:]
    div_y[0,:] = py[0,:]
    div_y[ny-1,:] = -py[ny-2,:]
    # return output divergence
    return div_x + div_y

In [20]:
# Tikhonov based image denoising (see TP 2)
def gaussian_denoise(u0,lbda,niter,alpha,Ndisplay=20,video=False):

    # initialize local variables
    ny,nx = u0.shape
    u = np.copy(u0)
    gx,gy = grad(u)
    E = np.zeros([niter+1,1])
    E[0] = np.sum((u-u0)**2 + lbda*(gx**2+gy**2))
   
    # initialize display (if needed)
    if video:
        plt.figure(figsize=(7,7))
        plt.imshow(u,cmap='gray')
        plt.title("itération 0: E = %.10e" % E[0])
    
    #############
    # main loop #
    #############
    for iter in range(1,niter+1):
        
        # update image u
        g = 2*(u-u0) - 2*lbda*div(gx,gy)
        u = u - alpha*g
        
        # update image gradient
        gx,gy = grad(u)
        
        # compute energy of u
        E[iter] = np.sum((u-u0)**2 + lbda*(gx**2+gy**2))
        
        # update display
        if video and (0 == iter%Ndisplay):
            plt.imshow(u,cmap='gray')
            display.clear_output(wait=True)
            display.display(pl.gcf())
            plt.title("itération %d : E = %.10e" % (iter,E[iter]))
            time.sleep(.1)
    
    # return outputs
    return u,E

## Exercice 1 : débruitage d’images par minimisation de la variation totale 

Étant donnée une image bruitée $u_0 : \Omega\to\mathbb{R}$, on s'intéresse au problème
de débruitage

$\DeclareMathOperator*{\argmin}{argmin~}$
\begin{equation}
\overline{u} = \argmin_{u : \Omega \to \mathbb{R}} \underset{= E(u)}{\underbrace{\frac{1}{2}\|u-u_0\|_2^2 + \lambda \mathrm{TV}(u)}}\,,\qquad (\mathcal{P})
\end{equation}

où $\lambda > 0$ et
$$\mathrm{TV}(u) = \displaystyle{\sum_{(x,y) \in \Omega}} \|(\nabla u)
(x,y)\|_2\,.$$


On admet que 
$\overline{u} = u_0 - \lambda \overline{w}$ où $\overline{w}$ désigne la
solution du _problème dual_

$$
  \overline{w} = \argmin_{w \in \mathcal{C}} \frac{1}{2}\|w - (u_0/\lambda)\|_2^2 \,,\qquad (\mathcal{P}^\star)
  %
$$

où

$$
  \mathcal{C} := \left\{ -\mathrm{div}_d(p) ~\left/~ p : \Omega\to \mathbb{R}^2 \text{
        et } \max_{(k,\ell)\in\Omega} \|p(k,\ell)\|_2 \leq 1 \right.\right\}\,.
  %
$$


1-a) Montrer que les problèmes $(\mathcal{P})$ et $(\mathcal{P}^\star)$ admettent chacun une unique solution.

1-b) On admet pour le moment que l'algorithme du gradient projeté appliqué au
  problème dual $(\mathcal{P}^\star)$ correspond au schéma itératif

$$
    \left\{
      \begin{array}{ccc}
        u^n &= &\lambda \cdot \mathrm{div}(p^n) + u_0 \\[10pt]
        \forall (k,\ell) \in\Omega\,,\quad p^{n+1}(k,\ell) &= &\dfrac{p^n(k,\ell) + \alpha/\lambda (\nabla u^n)(k,\ell)}{\max{\left(1,\|p^n(k,\ell) + \alpha/\lambda (\nabla u^n)(k,\ell)\|_2\right)}}\,,
      \end{array}
    \right.\qquad(\mathcal{S})
$$

Écrire une fonction `tvdenoise(u0,lambda,alpha,niter,...)` permettant d'approcher une solution de $(\mathcal{P})$ en implémentant ce schéma (renvoyer également en sortie la suite de l'énergie des itérés $(E(u^n))_{n \geq 0}$ générés par ce schéma.


1-c) Tester l'algorithme sur l'image `artifice.tiff` dégradée par un bruit additif blanc gaussien d'écart-type $\sigma = 20$ avec le réglage $\alpha = 0.2$ et $\lambda = 10$. Afficher l'image avant/après débruitage, ainsi que l'évolution de l'énergie au fil des itérations.

1-d) Comparer cet algorithme de débruitage à celui développé lors du TP n$^\circ$2 (méthode de Tikhonov pour le débruitage gaussien, le code développé en TP2 est rappelé en début de ce TP). 

**NB** : Afin d'effectuer une comparaison équitable de ces deux algorithmes, vous optimiserez pour chaque algorithme le réglage du paramètre $\lambda$ afin de minimiser l'écart quadratique entre l'image restaurée et l'image sans bruit.

1-e) Quels sont les avantages et inconvénients du débruitage par minimisation de la variation totale par rapport au débruitage par la méthode de Tikhonov (ne pas hésiter à zoomer sur les images)?

## Exercice 2 : construction du schéma dual pour la minimisation de la variation totale


Le but de cet exercice est de démontrer que le schéma numérique $(\mathcal{S})$ proposé en Exercice 1
correspond effectivement à un schéma de gradient projeté appliqué au problème
dual $(\mathcal{P}^\star)$.

2-a) En remarquant que $\mathcal{C} = - \mathrm{div}(\mathcal{B})$ où $\mathcal{B} = \left\{p : \Omega\to \mathbb{R}^2 ~\left/~ \max_{(k,\ell)\in\Omega} \|p(k,\ell)\|_2 \leq 1 \right.  \right\}$, montrer que la solution du problème duale $(\mathcal{P}^\star)$ satisfait

$$
    \overline{w} = -\mathrm{div}(\overline{p}) \quad \text{où} \quad
    \overline{p} = \argmin_{p \in \mathcal{B}} \frac{1}{2} \|\mathrm{div}{(p)} +
    (u_0/\lambda)\|_2^2\, \qquad (\mathcal{P}')
$$


2-b) En notant $J(p) = \frac{1}{2} \|\mathrm{div}{(p)} + u_0/\lambda\|_2^2$ et $\Pi_{\mathcal{B}}$ la projection sur le convexe $\mathcal{B}$, l'algorithme du gradient projeté appliqué au problème $(\mathcal{P}')$ correspond au schéma itératif

$$
p^{n+1} = \Pi_{\mathcal{B}}(p^n - \tau \nabla J(p^n))
$$

Calculer $\nabla J(p^n)$ et montrer que le schéma peut s'écrire

$$
\left\{
    %
    \begin{array}{cl}
      %
      u^n &= \lambda \cdot \mathrm{div}(p^n) + u_0 \\
      p^{n+1} &= \Pi_{\mathcal{B}}\left(p^n + \dfrac{\tau}{\lambda} \cdot \nabla u^n\right)
      %
    \end{array}
    %
\right.
$$


2-c) Expliciter $\Pi_{\mathcal{B}}$ afin de retrouver le schéma $(\mathcal{S})$